In [1]:
import re, os, sys
from functools import reduce
from itertools import combinations as comb, permutations as perm, combinations_with_replacement as combr
from operator import itemgetter
from pprint import pprint
from math import *
from collections import defaultdict
import networkx as nx
import numpy as np

def dprint(d):
    pprint(dict(d))
def lmap(*args, **kwargs):
    return list(map(*args, **kwargs))
def lmapi(*args, **kwargs):
    return list(map(int, *args, **kwargs))
def enum(*args, **kwargs):
    return enumerate(*args, **kwargs)

In [85]:
input_fname = "../inputs/day15-demo.in"

In [63]:
input_fname = "../inputs/day15.in"

In [86]:
def reread():
    lines = []
    with open(input_fname) as ifile:
        def r(): return ifile.readline().strip()
        def rs(): return ifile.readline().strip().split()
        def ri(): return int(ifile.readline().strip())
        def ris(): return list(map(int, ifile.readline().strip().split()))

        minx = miny = 1000
        maxx = maxy = 0
        for line in ifile:
            line = line.replace("Sensor at x=", "") \
                       .replace(", y=", " ") \
                       .replace(": closest beacon is at x=", " ")
            sensorx, sensory, beaconx, beacony = lmapi(line.split())
            minx = min(minx, sensorx, beaconx)
            maxx = max(maxx, sensorx, beaconx)
            miny = min(miny, sensory, beacony)
            maxy = max(maxy, sensory, beacony)
            lines.append(((sensorx, sensory), (beaconx, beacony)))

    return minx, maxx, miny, maxy, lines

minx, maxx, miny, maxy, lines = reread()
lines[:10]

[((2, 18), (-2, 15)),
 ((9, 16), (10, 16)),
 ((13, 2), (15, 3)),
 ((12, 14), (10, 16)),
 ((10, 20), (10, 16)),
 ((14, 17), (10, 16)),
 ((8, 7), (2, 10)),
 ((2, 0), (2, 10)),
 ((0, 11), (2, 10)),
 ((20, 14), (25, 17))]

## Part 1

In [31]:
minx, maxx, miny, maxy

(-645204, 3983949, 1000, 4169166)

In [32]:
# ar = np.zeros((maxx-minx+1, maxy-miny+1), dtype=int)
intervals = []
Y = 10
Y = 2000000
for (sx, sy), (bx, by) in lines:
    d = abs(sx - bx) + abs(sy - by)
    coverw = d - abs(sy - Y)
    if coverw >= 0:
        intervals.append((sx-coverw, sx+coverw+1))
minintx = min(map(itemgetter(0), intervals))
maxintx = max(map(itemgetter(1), intervals))

# print(minintx, intervals)
ar = np.zeros(maxintx-minintx+1, dtype=int)
for l, r in intervals:
    ar[l-minintx: r-minintx] = 1
for _, (bx, by) in lines:
    if by == Y:
        ar[bx] = 0
ar.sum()

4724228

## Part 2

In [68]:
from tqdm import trange

In [83]:
maxdim = 20 + 1
maxdim = 4000000 + 1
ar = np.zeros(maxdim, dtype=int)
ds = [abs(sx - bx) + abs(sy - by) for (sx, sy), (bx, by) in lines]
for Y in trange(maxdim):
    ar.fill(1)
    for ((sx, sy), _), d in zip(lines,ds):
        # d = abs(sx - bx) + abs(sy - by)
        coverw = d - abs(sy - Y)
        if coverw >= 0:
            # print("found", Y, coverw, sx-coverw, sx+coverw+1)
            ar[max(sx-coverw,0): sx+coverw+1] = 0
    if np.count_nonzero(ar):
        miss = list(zip(*np.nonzero(ar)))
        missx = miss[0][0]
        print(missx, Y, missx*4000000 + Y)
        break
# print(miss)
# missy, missx = miss[0]
# print(missx * 4000000 + missy)


  0%|          | 775/4000001 [00:09<13:26:30, 82.64it/s]


KeyboardInterrupt: 

In [87]:
maxdim = 20 + 1
# maxdim = 4000000 + 1
ar = np.zeros((maxdim, maxdim), dtype=int)
ds = [abs(sx - bx) + abs(sy - by) for (sx, sy), (bx, by) in lines]
for Y in trange(maxdim):
    # ar.fill(1)
    for ((sx, sy), _), d in zip(lines,ds):
        # d = abs(sx - bx) + abs(sy - by)
        coverw = d - abs(sy - Y)
        if coverw >= 0:
            # print("found", Y, coverw, sx-coverw, sx+coverw+1)
            ar[Y,max(sx-coverw,0): sx+coverw+1] += 1
    # if np.count_nonzero(ar):
    #     miss = list(zip(*np.nonzero(ar)))
    #     missx = miss[0][0]
    #     print(missx, Y, missx*4000000 + Y)
    #     break
# print(miss)
# missy, missx = miss[0]
# print(missx * 4000000 + missy)
ar


100%|██████████| 21/21 [00:00<00:00, 12750.49it/s]


array([[1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 1, 2, 2, 2, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 2, 2, 1, 3, 2, 3, 1, 1, 1, 1],
       [1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 4, 2, 2, 1, 1, 1],
       [1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 4, 1, 2, 2, 2, 1, 1],
       [1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 3, 2, 2, 1, 2, 2, 2, 1],
       [2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 3],
       [2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 3, 3],
       [3, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 3],
       [1, 3, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1],
       [1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1],
       [1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 0, 2, 2, 2, 1, 1, 1],
       [1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 3, 1, 2, 1, 1, 1, 1],
       [2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 3, 3, 3, 1, 1, 1,

In [88]:
from matplotlib import pyplot as plt

In [93]:
for row in ar:
    print(" ".join(map(str,row)))
    print()

1 1 1 1 1 1 2 2 2 2 2 1 2 1 2 1 1 1 1 1 1
1 1 1 1 1 2 2 2 2 2 2 3 1 2 2 2 1 1 1 1 1
1 1 1 1 2 2 2 2 2 2 3 2 2 1 3 2 3 1 1 1 1
1 1 1 2 2 2 2 2 2 2 1 2 2 3 2 4 2 2 1 1 1
1 1 2 2 2 2 2 2 2 1 1 1 2 2 4 1 2 2 2 1 1
1 2 2 2 2 2 2 2 1 1 1 1 1 3 2 2 1 2 2 2 1
2 2 2 2 2 2 2 1 1 1 1 1 2 2 2 2 2 1 2 2 3
2 2 2 2 2 2 1 1 1 1 1 2 2 2 2 2 2 2 1 3 3
3 2 2 2 2 1 1 1 1 1 1 1 2 2 2 2 2 1 2 2 3
1 3 2 2 1 1 1 1 1 1 1 1 1 2 2 2 1 2 2 2 1
1 1 3 1 1 1 1 1 1 1 1 1 2 1 2 1 2 2 2 1 1
1 1 2 2 1 1 1 1 1 1 1 2 2 2 0 2 2 2 1 1 1
1 2 2 1 1 1 1 1 1 1 2 2 2 1 3 1 2 1 1 1 1
2 2 1 1 1 1 1 1 1 2 2 2 1 3 3 3 1 1 1 1 1
2 1 1 1 1 1 1 1 2 2 2 1 3 3 3 3 3 2 1 1 1
1 1 1 1 1 1 1 1 1 3 1 2 2 3 3 3 3 3 2 1 1
1 1 1 1 1 1 1 1 2 1 4 2 2 2 3 3 3 3 3 2 1
1 1 1 1 1 1 1 1 1 3 2 3 2 2 2 3 3 3 3 3 2
1 1 1 1 1 1 1 1 2 2 2 2 3 2 2 2 3 3 3 2 2
1 1 1 1 1 1 1 2 2 1 1 2 3 3 2 2 2 3 2 2 2
1 1 1 1 1 1 2 2 1 1 1 2 3 3 3 2 2 1 2 2 2
